<a href="https://colab.research.google.com/github/RAHATHASSAN/microsoft16/blob/main/rp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets scikit-learn torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 32.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [2]:
# 2. Upload your dataset (zip)
# ================================
from google.colab import files
uploaded = files.upload()  # এখানে zip ফাইল সিলেক্ট করো (যেমন "new dataset with binary classification - Sheet1.zip")

import zipfile, os, glob

Saving cleaned_dataset_no_duplicates.zip to cleaned_dataset_no_duplicates.zip


In [3]:
# Extract zip
zip_path = list(uploaded.keys())[0]
extract_dir = "/content/dataset"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(extract_dir)


In [4]:
# Find CSV or Excel file
csv_files = glob.glob(os.path.join(extract_dir, "*.csv"))
excel_files = glob.glob(os.path.join(extract_dir, "*.xlsx")) + glob.glob(os.path.join(extract_dir, "*.xls"))

data_file = None
if csv_files:
    data_file = csv_files[0]
elif excel_files:
    data_file = excel_files[0]
else:
    raise FileNotFoundError("No CSV or Excel file found inside the zip!")

print("Dataset file found:", data_file)


Dataset file found: /content/dataset/cleaned_dataset_no_duplicates.csv


In [5]:
# 3. Load data
# ================================
import pandas as pd

if data_file.endswith(".csv"):
    df = pd.read_csv(data_file)
else:
    df = pd.read_excel(data_file)

print("Columns in dataset:", df.columns)
df.head()


Columns in dataset: Index(['ID', 'TEXT', 'DATASET', 'LANGUAGE', 'LABLE', 'LABLE.1'], dtype='object')


,ID,TEXT,DATASET,LANGUAGE,LABLE,LABLE.1
0,379,The evening was so great.,SMHD,English,Not Depressed,0
1,1160,I feel like I'm constantly battling myself.,SMHD,English,Depressed,1
2,856,আমি নিজেকে বোঝাতে পারি না।,BSMDD,Bangla,Depressed,1
3,1201,I'm trying to find my inner peace and strength.,SMHD,English,Not Depressed,0
4,251,The sunset was so calming.,SMHD,English,Not Depressed,0


In [6]:
# 4. Set column names
# Change these if needed
# ================================
TEXT_COL = "TEXT"    # তোমার dataset-এর টেক্সট কলামের নাম
LABEL_COL = "LABLE.1"  # তোমার dataset-এর লেবেল কলামের নাম (0/1)

# Drop NA and ensure types
df = df[[TEXT_COL, LABEL_COL]].dropna()
df[TEXT_COL] = df[TEXT_COL].astype(str)
df[LABEL_COL] = df[LABEL_COL].astype(int)

In [7]:
# 5. Train/Validation/Test Split
# ================================
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df[LABEL_COL], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df[LABEL_COL], random_state=42)

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")


Train: 880, Val: 110, Test: 111


In [8]:
# 6. Convert to HuggingFace Dataset
# ================================
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

In [9]:
# 7. Load tokenizer and model
# ================================
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

MODEL_NAME = "xlm-roberta-base"  # চাইলে "xlm-roberta-large" ব্যবহার করতে পারো
tokenizer = XLMRobertaTokenizer.from_pretrained(MODEL_NAME)
model = XLMRobertaForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# 8. Tokenize
# ================================
def tokenize(batch):
    return tokenizer(batch[TEXT_COL], padding="max_length", truncation=True, max_length=128)

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.rename_column(LABEL_COL, "labels")
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/880 [00:00<?, ? examples/s]

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Map:   0%|          | 0/111 [00:00<?, ? examples/s]

In [11]:
# 9. Training setup
# ================================
from transformers import TrainingArguments, Trainer
# from transformers.training_args import EvaluationStrategy # Corrected import # Removed import
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds)
    }

training_args = TrainingArguments(
    output_dir="./results",
    # evaluation_strategy="epoch", # Changed to string literal # Removed evaluation strategy
    save_strategy="epoch",
    # do_eval=True, # Added to enable evaluation # Removed do_eval
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    # load_best_model_at_end=True, # Removed load_best_model_at_end
    # metric_for_best_model="f1" # Removed metric_for_best_model
    report_to="none" # Disable wandb and other reporting
)

In [12]:
# 10. Trainer
# ================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-4070264903.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
# 11. Train
# ================================
trainer.train()

Step,Training Loss


TrainOutput(global_step=165, training_loss=0.287100820830374, metrics={'train_runtime': 4210.4123, 'train_samples_per_second': 0.627, 'train_steps_per_second': 0.039, 'total_flos': 173653296537600.0, 'train_loss': 0.287100820830374, 'epoch': 3.0})

In [14]:
# 12. Evaluate
# ================================
results = trainer.evaluate(dataset["test"])

print("Evaluation Results:")
print(f"  Accuracy: {results['eval_accuracy']:.4f}")
print(f"  F1 Score: {results['eval_f1']:.4f}")
print(f"  Precision: {results['eval_precision']:.4f}")
print(f"  Recall: {results['eval_recall']:.4f}")

Evaluation Results:
  Accuracy: 0.9820
  F1 Score: 0.9815
  Precision: 1.0000
  Recall: 0.9636


In [20]:
# 13. Prediction Example
# ================================
example_text = "Happiness is a journey, not a destination. Enjoying the ride!"
inputs = tokenizer(example_text, return_tensors="pt", truncation=True, padding=True)
outputs = model(**inputs)
pred = outputs.logits.argmax(dim=1).item()
print("Prediction:", "Depressed" if pred == 1 else "Not Depressed")

Prediction: Not Depressed
